# Imports

In [1]:
import os
import re
import math
import requests
import numpy as np
import pandas as pd
import pandas.io.sql as sqlio
from time import sleep
from random import randint
from bs4 import BeautifulSoup
from datetime import datetime
from sqlalchemy import create_engine

## Get infos for one city - Düsseldorf

In [7]:
# only wohnung

url = 'https://www.immonet.de/immobiliensuche/sel.do?parentcat=1&objecttype=1&pageoffset=378&listsize=27&suchart=1&sortby=0&city=100207&marketingtype=2&page=1'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.text, "html.parser")

# Find number of rent offers.
a = soup.findAll('h1', class_='box-50')[0].get_text()
total_offers = int(re.search('\d+', a).group())

## Get all offers ids for Düsseldorf - only Wohnung (apartments)

In [493]:
offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

# Get the number of pages to scrape - rounded to down
number_of_pages = math.floor(total_offers / len(offers_list_1page))

# City code
city_code = "100207" # Düsseldorf

# List of all ids
ids_list = []


for page in range(number_of_pages):
    
    url = f"https://www.immonet.de/immobiliensuche/sel.do?parentcat=1&objecttype=1&pageoffset=1&listsize=26&suchart=1&sortby=0&city={city_code}&marketingtype=2&page={page}"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

    for i in range(len(offers_list_1page)):
        try:
            ids_list.append(offers_list_1page[i].find('a')['data-object-id'])
        except:
            pass
        #sleep(randint(1, 4))

In [494]:
len(ids_list)

366

In [495]:
len(set(ids_list))

341

## get infos for each offer_id

In [2]:
'''
Here we will get the infos from the info panels for each offer.
'''
# City code
city_code = "100207"
city = 'Dusseldorf'

infos_list = []
now = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

for Id in set(ids_list):

    url = f"https://www.immonet.de/angebot/{Id}?drop=sel&related=false&product=standard"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    panels_infos = soup.findAll('div', class_='row box-50')
    # get all infos about each offer (all infos are mixed)
    
    offer_infos = []
    for panel in panels_infos:
        offer_infos.append(panel.text.replace('\n', '').replace('\t', '-'))
    
    infos_list.append({'offer_id': Id, 'datetime': now, 'city': city, 'city_code': city_code, 'infos': offer_infos})
    
    if len(offer_infos) == 0:
        print(Id, ' - ERROR')    

NameError: name 'ids_list' is not defined

In [498]:
df_raw = pd.DataFrame(infos_list)

In [499]:
df_raw.head()

,offer_id,datetime,infos
0,45383090,20211003-191642,[-----WOHNEN UNKOMPLIZIERT - EIN PREIS ALLES D...
1,45155366,20211003-191642,[-----Helle 2 Zimmer Wohnung mit Dachterrasse-...
2,45013603,20211003-191642,[-----Voll möbliertes Apartment mit hotelähnli...
3,45299018,20211003-191642,"[-----Düsseldorf-Bilk! Ruhige Seitenstraße, 2 ..."
4,45382436,20211003-191642,"[-----Helle 3-Zimmerwohnung mit Balkon, Wannen..."


In [481]:
df_raw['infos'][0]

['-----Unterrath: Helles Appartement am Kartäuserpark mit Einbauküche, modernisiertem Tageslichtbad und Balkon----------------Apartment ------mieten-------------------in Düsseldorf-----------------Kartäuserstrasse   . .------------40468\xa0------Düsseldorf------------Auf Karte anzeigenDieses Angebot teilenDieses Angebot teilen',
 'Meine Orte',
 'Preise & Kosten----------Miete zzgl. NK----------525.0\xa0€---------------------Miete inkl. NK----------625.0\xa0€---------------------Nebenkosten----------100.0\xa0€-----------Heizkosten in Nebenkosten enthalten:Ja----------Kaution/ Genossenschaftsanteile----------1575.0\xa0€-----------',
 'Größe & Zustand-----------------------Zimmer--------------------1.0------Wohnfläche ca.------40.0\xa0m²Zustand-------Teil- oder vollsaniert------Baujahr1968Verfügbar ab--------------sofort-------------',
 'EnergieEnergieeffizienzklasseKlasse F----------------------------Endenergieverbrauch--------------------------188.0 kWh/(m²*a)--------Energieausweis-----

## Get offers for houses too

In [22]:
# get all offers quantity - haus und wohnung

total_offers = 0

for i in range(1,3):
    url = f'https://www.immonet.de/immobiliensuche/sel.do?parentcat={i}&objecttype=1&pageoffset=378&listsize=27&suchart=1&sortby=0&city=100207&marketingtype=2&page=1'
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    # Find number of rent offers.
    a = soup.findAll('h1', class_='box-50')[0].get_text()
    total_offers += int(re.search('\d+', a).group())

In [23]:
# get all offers ids for haus und wohnung in Dusseldorf

offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

# Get the number of pages to scrape - rounded to down
number_of_pages = math.floor(total_offers / len(offers_list_1page))

# City code
city_code = "100207" # Düsseldorf
city = 'Dusseldorf'

# List of all ids
ids_list = []

# wohnung/haus code
l_opt = [1, 2]

for opt in l_opt:
    for page in range(number_of_pages):
        url = f"https://www.immonet.de/immobiliensuche/sel.do?parentcat={opt}&objecttype=1&pageoffset=1&listsize=26&suchart=1&sortby=0&city={city_code}&marketingtype=2&page={page}"
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, "html.parser")
    
        offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

        for i in range(len(offers_list_1page)):
            try:
                if opt == 1:
                    ids_list.append({ 'offer_id': offers_list_1page[i].find('a')['data-object-id']'city': city, 'city_code': city_code, 'type': 'wohnung'})
                if opt == 2:
                    ids_list.append({ 'offer_id': offers_list_1page[i].find('a')['data-object-id'], 'city': city, 'city_code': city_code, 'type': 'haus'})
            except:
                pass
            
# Create a dataframe with the infos
df_ids = pd.DataFrame(ids_list)

In [29]:
'''
Here we will get the infos from the info panels for each offer.
'''

infos_list = []
now = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')

for Id in set(df_ids['offer_id']):

    url = f"https://www.immonet.de/angebot/{Id}?drop=sel&related=false&product=standard"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    panels_infos = soup.findAll('div', class_='row box-50')
    # get all infos about each offer (all infos are mixed)
    
    offer_infos = []
    for panel in panels_infos:
        offer_infos.append(panel.text.replace('\n', '').replace('\t', '-'))
    
    infos_list.append({'offer_id': Id, 'datetime': now, 'city': city, 'city_code': city_code, 'infos': offer_infos})
    
    if len(offer_infos) == 0:
        print(Id, ' - ERROR')   
        
# create a dataframe
df_raw_infos = pd.DataFrame(infos_list)

In [30]:
df_raw_infos = pd.DataFrame(infos_list)

In [32]:
df_raw_infos.shape

(438, 5)

## Get offers for more cityes

In [2]:
# get main German cities codes

cities_dict = {
    'Dusseldorf': 100207,
    'Berlin': 87372,
    'Essen': 102157,
    'Munchen': 121673,
    'Koln': 113144,
    'Stuttgart': 143262,
    'Dresden': 100051,
    'Hannover': 109489,
    'Dortmund': 99990,
    'Frankfurt am Main': 105043,
    'Hamburg': 109447
}
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
now2 = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
# get all offers quantity - haus und wohnung

cities_offers = []

for city, code in cities_dict.items():
    
    total_offers = 0
    
    for i in range(1,3):
        url = f'https://www.immonet.de/immobiliensuche/sel.do?parentcat={i}&objecttype=1&pageoffset=378&listsize=27&suchart=1&sortby=0&city={code}&marketingtype=2&page=1'
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
        page = requests.get(url, headers=headers)
        soup = BeautifulSoup(page.text, "html.parser")
        
        # Find number of rent offers.
        a = soup.findAll('h1', class_='box-50')[0].get_text()
        total_offers += int(re.search('\d+', a).group())
    
    cities_offers.append({'extraction_datetime': now, 'city': city, 'city_code': code, 'offers': total_offers})
    
offers_by_page = len(soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter"))    
df_offers = pd.DataFrame(cities_offers)
df_offers.to_csv(f'../data/total_offers_by_city_{now2}.csv', index=False)

In [3]:
df_offers.head()

,extraction_datetime,city,city_code,offers
0,2021-10-12 11:10:28,Dusseldorf,100207,409
1,2021-10-12 11:10:28,Berlin,87372,498
2,2021-10-12 11:10:28,Essen,102157,570
3,2021-10-12 11:10:28,Munchen,121673,804
4,2021-10-12 11:10:28,Koln,113144,198


In [67]:
df_offers.shape

(11, 4)

In [4]:
# List of all ids
ids_list = []
offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
now2 = datetime.now().strftime('%Y_%m_%d_%H_%M_%S')

for c in range(len(df_offers)):
    city = df_offers.loc[c]['city']
    city_code = df_offers.loc[c]['city_code']
    offers = df_offers.loc[c]['offers']
    
    # get all offers ids for haus und wohnung in each city
    

    # Get the number of pages to scrape - rounded to down
    number_of_pages = math.floor(offers / offers_by_page)
    
    # wohnung/haus code
    l_opt = [1, 2]

    for opt in l_opt:
        for page in range(number_of_pages):
            url = f"https://www.immonet.de/immobiliensuche/sel.do?parentcat={opt}&objecttype=1&pageoffset=1&listsize=26&suchart=1&sortby=0&city={city_code}&marketingtype=2&page={page}"
            headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
            page = requests.get(url, headers=headers)
            soup = BeautifulSoup(page.text, "html.parser")

            offers_list_1page = soup.findAll('div', class_="col-xs-12 place-over-understitial sel-bg-gray-lighter")

            for i in range(len(offers_list_1page)):
                try:
                    if opt == 1:
                        ids_list.append({'extraction_datetime': now, 'offer_id': offers_list_1page[i].find('a')['data-object-id'], 'city': city, 'city_code': city_code, 'type': 'wohnung'})
                    if opt == 2:
                        ids_list.append({'extraction_datetime': now, 'offer_id': offers_list_1page[i].find('a')['data-object-id'], 'city': city, 'city_code': city_code, 'type': 'haus'})
                except:
                    pass
            sleep(randint(1, 2)) 
    sleep(randint(1, 5))          
            
# Create a dataframe with the infos
df_ids = pd.DataFrame(ids_list)
df_ids.drop_duplicates(subset='offer_id', inplace=True)

# save as csv file
output_dir = '../data/'
filename = f'immonet_de_{now2}.csv'
df_ids.to_csv(os.path.join(output_dir, filename), index=False)

In [5]:
df_ids.head()

,extraction_datetime,offer_id,city,city_code,type
0,2021-10-12 11:10:55,45627685,Dusseldorf,100207,wohnung
1,2021-10-12 11:10:55,45366691,Dusseldorf,100207,wohnung
2,2021-10-12 11:10:55,45578967,Dusseldorf,100207,wohnung
3,2021-10-12 11:10:55,45475784,Dusseldorf,100207,wohnung
4,2021-10-12 11:10:55,45466649,Dusseldorf,100207,wohnung


In [7]:
df_ids.shape

(7375, 5)

In [41]:
# get all infor for all founded offers

'''
Here we will get the infos from the info panels for each offer.
'''
count = 0
#infos_list = []
#datetime_list = []
now = datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
now2 = datetime.now().strftime('%Y_%m_%d')

for Id in set(df_ids['offer_id']):

    url = f"https://www.immonet.de/angebot/{Id}?drop=sel&related=false&product=standard"
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
    page = requests.get(url, headers=headers)
    soup = BeautifulSoup(page.text, "html.parser")
    
    panels_infos = soup.findAll('div', class_='row box-50')
    # get all infos about each offer (all infos are mixed)
    
    offer_infos = []
    for panel in panels_infos:
        text = panel.text.replace('\n', '').replace('\t', '-')
        offer_infos.append(text)
        
    
    
    infos_list.append({'offer_id': Id,
                       'extraction_date': now,
                       'city': df_ids['city'][count],
                       'city_code': df_ids['city_code'][count],
                       'offer_type': df_ids['type'][count],
                       'offer_infos': offer_infos})
    #print(count)
    count += 1
    
    #if len(offer_infos) != 0:
    #    datetime_list.append(datetime.now().strftime('%Y-%m-%d-%H:%M:%S'))
    #    infos_list.append(offer_infos)
    #else:
    #    infos_list.append(np.nan)
    #    print('ERROR')

df_infos = pd.DataFrame(infos_list)
df_infos.drop_duplicates(subset='offer_id', inplace=True)
df_infos.to_csv(f'../data/df_infos_{now2}.csv', index=False)
#df_infos = df_ids.copy()
#df_infos['scrapy_time'] = datetime_list
#df_infos['infos'] = infos_list

In [66]:
df_infos.head()

,offer_id,extraction_date,city,city_code,offer_type,offer_infos
0,44782312,2021-10-12 11:10:55,Dusseldorf,100207,wohnung,['-----Neubau-Erstbezug: Elegante 3-Zimmer-Woh...
1,45521517,2021-10-12 11:10:55,Dusseldorf,100207,wohnung,['-----möblierte Terrassenwohnung mit Gartennu...
2,45556158,2021-10-12 11:10:55,Dusseldorf,100207,wohnung,"['-----Große, helle und sonnige 2-Zimmer Dachg..."
3,45605951,2021-10-12 11:10:55,Dusseldorf,100207,wohnung,['-----Bezugsfertige Etagenwohnung im Dortmund...
4,42868803,2021-10-12 11:10:55,Dusseldorf,100207,wohnung,['-----Erstbezug - Moderne Wohnungen----------...


In [65]:
df_infos.shape

(6753, 6)